## Hello! 
This is my submission for the intro assignment for Acoustic Species ID. Hope you enjoy! 

-TQ Zhang

In [30]:
# First step: import pandas to use in the submission
import pandas as pd
print("Let's get started!")

Let's get started!


In [31]:
# displaying dataframe for reference
pd.read_csv("Peru_2019_AudioMoth_Data_Full.csv")

/var/folders/sj/59v4c3dd1r1brq_2d_bjy80w0000gn/T/ipykernel_77376/860044312.py:2: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("Peru_2019_AudioMoth_Data_Full.csv")


,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_145649.WAV,GRABADOR-SDZG-AM-1,20190616_145649.WAV,1802240,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 14:56,2.346667,NaN,Recorded at 14:56:49 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
1,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_150000.WAV,GRABADOR-SDZG-AM-1,20190616_150000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:00,60.000469,NaN,Recorded at 15:00:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:11-08:00,WAV,WAV,audio/x-wav
2,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_151000.WAV,GRABADOR-SDZG-AM-1,20190616_151000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:10,60.000469,NaN,Recorded at 15:10:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:13-08:00,WAV,WAV,audio/x-wav
3,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_152000.WAV,GRABADOR-SDZG-AM-1,20190616_152000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:20,60.000469,NaN,Recorded at 15:20:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:15-08:00,WAV,WAV,audio/x-wav
4,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190616_153000.WAV,GRABADOR-SDZG-AM-1,20190616_153000.WAV,46080360,1.0,1.0,384000.0,768000.0,16.0,16.06.2019 15:30,60.000469,NaN,Recorded at 15:30:00 16/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:13:16-08:00,WAV,WAV,audio/x-wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98633,WWF-5,NaN,GRABADOR-WWF-5/5D3157B0.WAV,GRABADOR-WWF-5,5D3157B0.WAV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File is empty,NaN,NaN,2019:12:07 19:06:53-08:00,NaN,NaN,NaN
98634,WWF-5,NaN,GRABADOR-WWF-5/5D315A08.WAV,GRABADOR-WWF-5,5D315A08.WAV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File is empty,NaN,NaN,2019:12:07 18:45:59-08:00,NaN,NaN,NaN
98635,WWF-5,NaN,GRABADOR-WWF-5/5D315C60.WAV,GRABADOR-WWF-5,5D315C60.WAV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File is empty,NaN,NaN,2019:12:07 18:45:59-08:00,NaN,NaN,NaN
98636,WWF-5,NaN,GRABADOR-WWF-5/5D315EB8.WAV,GRABADOR-WWF-5,5D315EB8.WAV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File is empty,NaN,NaN,2019:12:07 18:45:59-08:00,NaN,NaN,NaN


The approach to writing a general function to perform an SRS with the restrictions given would be:

    1. Read the csv file into a dataframe
        a. Return False if the file was not found
    2. Remove unwanted rows 
        a. Devices with <24 total recordings
        b. Recordings with <60 sec durations
    3. Create column combining device code and starting hour of recording
    4. Take a stratified random sample, grouping by our new column and taking one random recording from each group, storing the sample in a dataframe
    5. Write the resulting dataframe to a .csv file
    6. Return True
So, the function should look like this:

In [32]:
def stratified_random_sample(path):
    # adjustable variables, in case the file we're reading uses a different column naming convention/requirements
    min_recordings = 24
    min_duration = 60
    code = "AudioMothCode"
    time = "StartDateTime"
    alt_time = "Comment"
    duration = "Duration"
    output_file_name = "submission.csv"
    exp_shape = (744, 20)
    # read file
    try:
        data = pd.read_csv(path)
    except FileNotFoundError:
        return False
    # remove rows
        # dropping rows of AMs with < 24 recordings
    num_recordings = data.get([code, time]).fillna(value = "dummy").groupby(code).count().reset_index()
    less_than_min = num_recordings[num_recordings.get(time) < min_recordings][code]
    data = data.set_index(code).drop(less_than_min).reset_index()
        # dropping recordings with duration < 60 sec
    data = data[data[duration] >= min_duration]
    # create new column
    data["StartHour"] = data[time].str[11:13]
    data["StartHour"] = data["StartHour"].fillna(value = data[alt_time].str[12:14]).dropna()
    data["AMC+Hour"] = data[code] + " " + data["StartHour"]
    # take SRS
    strat_sample = data.groupby("AMC+Hour", group_keys = False).apply(lambda x: x.sample(1)).sort_values(code)
    strat_sample = strat_sample.drop(["StartHour", "AMC+Hour"], axis = 1)
    # double check that we have 1 recording for every hour for every camera
    print(strat_sample.shape)
    if strat_sample.shape != exp_shape:
        return False
    # write to csv
    strat_sample.to_csv(output_file_name)
    # done!
    return True

stratified_random_sample("Peru_2019_AudioMoth_Data_Full.csv")

/var/folders/sj/59v4c3dd1r1brq_2d_bjy80w0000gn/T/ipykernel_77376/985730980.py:13: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


(744, 20)


True

Our Stratified Random Sample is now saved in submission.csv!

Notes: 

The function uses StartDateTime and Comment to determine the hour of the recording. It assumes the formatting of StartDateTime and Comment to be as follows: StartDateTime stored as

 ___**DD.MM.YYYY hh:mm**___
 
 and Comment containing the starting time, formatted as

 ___**Recorded at hh:mm:ss DD/MM/YYYY (UTC) by...**___
 
 A file with different formatting would not work. However, recordings without starting time data are not explicitly dropped.

## Thank you!

In [33]:
pd.read_csv("submission.csv")

,Unnamed: 0,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,...,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,634,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190621_003000.WAV,GRABADOR-SDZG-AM-1,20190621_003000.WAV,46080360,1.0,1.0,384000.0,...,16.0,21.06.2019 00:30,60.000469,NaN,Recorded at 00:30:00 21/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:33:00-08:00,WAV,WAV,audio/x-wav
1,193,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190617_230000.WAV,GRABADOR-SDZG-AM-1,20190617_230000.WAV,46080360,1.0,1.0,384000.0,...,16.0,17.06.2019 23:00,60.000469,NaN,Recorded at 23:00:00 17/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:19:08-08:00,WAV,WAV,audio/x-wav
2,764,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190621_221000.WAV,GRABADOR-SDZG-AM-1,20190621_221000.WAV,46080360,1.0,1.0,384000.0,...,16.0,21.06.2019 22:10,60.000469,NaN,Recorded at 22:10:00 21/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:37:03-08:00,WAV,WAV,audio/x-wav
3,2053,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190630_210000.WAV,GRABADOR-SDZG-AM-1,20190630_210000.WAV,46080360,1.0,1.0,384000.0,...,16.0,30.06.2019 21:00,60.000469,NaN,Recorded at 21:00:00 30/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 04:17:33-08:00,WAV,WAV,audio/x-wav
4,2483,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190703_204000.WAV,GRABADOR-SDZG-AM-1,20190703_204000.WAV,46080360,1.0,1.0,384000.0,...,16.0,03.07.2019 20:40,60.000469,NaN,Recorded at 20:40:00 03/07/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 04:31:09-08:00,WAV,WAV,audio/x-wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,96610,WWF-5,NaN,GRABADOR-WWF-5/5D1EA368.WAV,GRABADOR-WWF-5,5D1EA368.WAV,46080192,1.0,1.0,384000.0,...,16.0,NaN,60.000250,NaN,Recorded at 01:10:00 05/07/2019 (UTC) by Audio...,NaN,2019:12:07 16:12:44-08:00,WAV,WAV,audio/x-wav
740,97758,WWF-5,NaN,GRABADOR-WWF-5/5D292608.WAV,GRABADOR-WWF-5,5D292608.WAV,46080192,1.0,1.0,384000.0,...,16.0,NaN,60.000250,NaN,Recorded at 00:30:00 13/07/2019 (UTC) by Audio...,NaN,2019:12:07 19:39:17-08:00,WAV,WAV,audio/x-wav
741,96882,WWF-5,NaN,GRABADOR-WWF-5/5D2120E8.WAV,GRABADOR-WWF-5,5D2120E8.WAV,46080192,1.0,1.0,384000.0,...,16.0,NaN,60.000250,NaN,Recorded at 22:30:00 06/07/2019 (UTC) by Audio...,NaN,2019:12:07 18:54:37-08:00,WAV,WAV,audio/x-wav
742,97383,WWF-5,NaN,GRABADOR-WWF-5/5D25B720.WAV,GRABADOR-WWF-5,5D25B720.WAV,46080192,1.0,1.0,384000.0,...,16.0,NaN,60.000250,NaN,Recorded at 10:00:00 10/07/2019 (UTC) by Audio...,NaN,2019:12:07 17:56:06-08:00,WAV,WAV,audio/x-wav
